In [1]:
import pandas as pd
import numpy as np
import os
import math

获取所需要的标签

In [2]:
columns_ls = ['Epitope - Name',
 'Assay - Qualitative Measurement',
 'Assay - Measurement Inequality',
 'Assay - Quantitative measurement',
 'Assay - Units',
 'MHC Restriction - Name',
 'MHC Restriction - Class',
#  'Type',
 'Reference - Date',
#  'Title',
#  'Method/Technique',
 'Assay - Response measured',
 'Host - Name']

new_columns_ls = ['Description',
 'Qualitative Measure',
 'Measurement Inequality',
 'Quantitative measurement',
 'Units',
 'Allele Name',
 'MHC allele class',
#  'Ref Type',
 'Ref Date',
#  'Ref Title',
#  'Assay Method/Technique',
 'Assay Group',
 'Host Name']

 # 输入与输出路径
# IEDB_origin = '/raid/hly/PK-panAllele/data/mhc_ligand_full.csv' 
# IEDB_ori_need = '/raid/hly/PK-panAllele/data/IEDB2022_need.csv'
IEDB_origin = '/mnt/zt/Dopaap/original_data/2021_mhc_ligand_table_export_1708357594.csv'
IEDB_ori_need = '/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/2021_IEDB_need_column.csv'

In [3]:
# 单表头数据header=0,双表头数据header=1
df = pd.read_csv(IEDB_origin,header=0)

/home/zt/anaconda3/envs/molmap/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning: Columns (12,13,14,27,28,29,30,34,35,36,37,38,39,40,41,42,45,46,47,48,49,51,52,53,54,55,56,65,66,67,68,72,73,74,75,76,79,80,81,82,83,84,85,86,87,88,89,92,95,100,101) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 获取原始文件中需要的标签。存至“IEDB_ori_need_Data”
df_need = df[columns_ls]
df_need.columns = [new_columns_ls]
df_need.to_csv(IEDB_ori_need,index=0)

删除不需要的数据

In [5]:
#读取文件
df_all = pd.read_csv('/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/2021_IEDB_need_column.csv')

/home/zt/anaconda3/envs/molmap/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3257: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# 筛选出host为人类的条目--Homo sapiens (human) or human（Homo sapiens）
df_host = df_all[(df_all['Host Name'] == 'Homo sapiens (human)') | (df_all['Host Name'] =='human (Homo sapiens)') ]

# 筛选出有measurement标签的数值
df_QM = df_host[pd.notnull(df_host['Quantitative measurement'])]

#保留规范化的多肽序列（不含符号且不含'BXJZ'）
    # 删除seq含'-'或' '的行
df1 = df_QM.drop(index=(df_QM.loc[(df_QM['Description'].str.contains('-'))].index))
df2 = df1.drop(index=(df1.loc[(df1['Description'].str.contains(' '))].index))
    # 删除seq含'BXJZ'的行
df3 = df2.drop(index=(df2.loc[(df2['Description'].str.contains('B'))].index))
df4 = df3.drop(index=(df3.loc[(df3['Description'].str.contains('X'))].index))
df5 = df4.drop(index=(df4.loc[(df4['Description'].str.contains('J'))].index))
df6 = df5.drop(index=(df5.loc[(df5['Description'].str.contains('Z'))].index))

# 筛选出精确的allele name（含'HLA'，'：'） 
df_QM = df6.loc[(df6['Allele Name'].str.contains('HLA'))]
df_QM = df_QM.loc[(df_QM['Allele Name'].str.contains(":"))]

# 删去assay group 不为 “KD”及 “IC50”，“EC50”相关的条目
    #删去 assay group为 3D structure 的条目
df_assay_1 = df_QM.drop(index=(df_QM.loc[(df_QM['Assay Group']== '3D structure' )].index))
    #删去 assay group为 50% dissociation temperature 的条目
df_assay_2 = df_assay_1.drop(index=(df_assay_1.loc[(df_assay_1['Assay Group']== "50% dissociation temperature" )].index))
    #删去 assay group为 half life 的条目
df_assay_3 = df_assay_2.drop(index=(df_assay_2.loc[(df_assay_2['Assay Group']== 'half life' )].index))
     #删去 assay group为 off rate 的条目
df_assay_4 = df_assay_3.drop(index=(df_assay_3.loc[(df_assay_3['Assay Group']== 'off rate' )].index))
    #删去 assay group为 on rate 的条目
df_assay_5 = df_assay_4.drop(index=(df_assay_4.loc[(df_assay_4['Assay Group']== 'on rate' )].index))
    #删去 assay group为 qualitative binding 的条目
df_assay_6 = df_assay_5.drop(index=(df_assay_5.loc[(df_assay_5['Assay Group']== 'qualitative binding' )].index))

#删掉unit不为nM的数据
df_assay_7 = df_assay_6.drop(index=(df_assay_6.loc[(df_assay_6['Units']!= 'nM' )].index))

# 删去'Measurement Inequality'为'>','<','>=','<='的行
df_MI_1 = df_assay_7.drop(index=(df_assay_7.loc[(df_assay_7['Measurement Inequality']== '>' )].index))
df_MI_2 = df_MI_1.drop(index=(df_MI_1.loc[(df_MI_1['Measurement Inequality']== '<' )].index))
df_MI_3 = df_MI_2.drop(index=(df_MI_2.loc[(df_MI_2['Measurement Inequality']== '>=' )].index))
df_MI_4 = df_MI_3.drop(index=(df_MI_3.loc[(df_MI_3['Measurement Inequality']== '<=' )].index))

# 去除活性值大于50000的条目
df_QM = df_MI_4.drop(index=(df_MI_4.loc[(df_MI_4['Quantitative measurement'] > 50000)].index))

# 去除“reference Title”为“Quantitating T cell cross-reactivity for unrelated peptide antigens.”的条目。
# df_QM = df_QM.drop(index=(df_QM.loc[(df_QM['Ref Title']== 'Quantitating T cell cross-reactivity for unrelated peptide antigens.' )].index))


In [7]:
# 数据标准化 【1-log10(IC50)/log10(50000 nM)】 ,IC50单位为nM
df_QM["Normalized_QM"] = 1-(df_QM["Quantitative measurement"].apply(np.log10))/np.log10(50000)

In [8]:
# 重置索引并保存文件
df_QM = df_QM.reset_index(drop = True)

df_QM.to_csv('/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/2021_IEDB_processed.csv') 

In [10]:
df_2022 = df_QM[df_QM['Ref Date'] >= 2022]
df_2022.to_csv('/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/2022_IEDB_processed.csv')

df_2023 = df_QM[df_QM['Ref Date'] >= 2023]
df_2023.to_csv('/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/2023_IEDB_processed.csv')

获得new arraylist 测试集

In [13]:
df_QM = pd.read_csv('/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/2023_IEDB_processed.csv',index_col=0)

In [14]:
out_dir = '/mnt/zt/Dopaap/calculate_webtools_score/newdatalist/data/testdata/2023'
for name, group in df_QM.groupby('Allele Name') :
    # 替换符号，如将/替换成&
    name = name.replace('/', '&')
    name = name.replace('*','_')
    name = name.replace(':','')
    pd.DataFrame(group).to_csv(os.path.join(out_dir,name+'.csv'))

In [ ]:
# 统计

#统计了各数据集的条目数，多肽长度的最值，活性值最值，标准化活性值的最值。

files_dir = '/raid/hly/PK-panAllele/data/train_MID_2'
files = os.listdir(files_dir)
statistic = []
for file in files:
    df = pd.read_csv(os.path.join(files_dir,file))
    file_name = file.split('.')[0]
    # 条目数
    pep_num = len(df)

    #多肽长度范围
    pep_len_min = min(map(len,df['Description']))
    pep_len_max = max(map(len,df['Description']))

    #QM范围
    QM_min = min(df['Quantitative measurement'])
    QM_max = max(df['Quantitative measurement'])
    
    #log_QM范围
    Normalized_QM_min = min(df['Normalized_QM'])
    Normalized_QM_max = max(df['Normalized_QM'])

    statistic.append([file_name,pep_num,pep_len_min,pep_len_max,QM_min,QM_max,Normalized_QM_min,Normalized_QM_max])
df_stt = pd.DataFrame(statistic,columns=['HLA-allele','pep_num','pep_len_min','pep_len_max','QM_min','QM_max','Normalized_QM_min','Normalized_QM_max'])
df_stt.sort_values(by='pep_num',axis=0,ascending=False, inplace=True)
df_stt = df_stt.reset_index(drop = True)
df_stt.to_csv('/raid/hly/PK-panAllele/data/train_MID_2_statistics220708.csv')
